<a href="https://colab.research.google.com/github/HighPriestCthulhu/IntroToTF/blob/master/Boston_Market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 0s 0us/step


In [3]:
test_data.shape

(102, 13)

In [0]:
#Find Z scores for all values in list to normalise data using feature-wise normalisation
mean = train_data.mean(axis=0)
train_data -=mean
std=train_data.std(axis=0)
train_data /= std
test_data -=mean
test_data /=std

In [0]:
from keras import models, layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(units=64,
                         activation='relu',
                         input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(units=64,
                         activation='relu')) #non-linear relu function delinearises the data and makes it possible to find complex patterns
  model.add(layers.Dense(units=1)) #defaults to purely linear function
  model.compile(optimizer='rmsprop',
               loss='mse',
               metrics=['mae'])
  return model 

  

In [13]:
import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 500
all_scores = []

for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i* num_val_samples:(i+1)*num_val_samples]
  val_targets = train_targets[i* num_val_samples:(i+1)*num_val_samples]
  
  partial_train_data = np.concatenate(
    [train_data[:i* num_val_samples],
     train_data[(i+1)* num_val_samples:]],
    axis=0)
  partial_train_targets = np.concatenate(
    [train_targets[:i* num_val_samples],
     train_targets[(i+1)* num_val_samples:]],
    axis=0)
  
  
  model = build_model()
  model.fit(partial_train_data, partial_train_targets, 
            epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)
  
print(all_scores)

processing fold # 2
processing fold # 3
[2.30608329914584, 2.8143377115230748, 2.8703539985241275, 2.403667432246822]
